In [ ]:
import json
import requests
import pandas as pd
import boto3
import csv 
from io import StringIO
import re
import datetime as dt
from datetime import datetime, timedelta
import time as t



insta_id = "insta_id"
token = "access_token"

# finding the cities where followers are

url_city = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"metric":"audience_city","period":"lifetime","access_token":""+token}
headers_city = {
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }
response = requests.request("GET", url_city, headers=headers_city, params=querystring)
cities = json.loads(response.text)

x = pd.DataFrame(list(cities['data'][0]['values'][0]['value'].items()))
x.columns = ['cities','followers']
y = x['cities'].apply(lambda x:x.split(','))
x['city'] = y.apply(lambda x:x[0])
x['state']= y.apply(lambda x:x[1])
x = x.drop(['cities'],axis=1)
new_dic = x.to_csv(index=False)

# # # finding the countries where followers are

url_country = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"metric":"audience_country","period":"lifetime","access_token":""+token}
headers_country = {
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url_country, headers=headers_country, params=querystring)

countries = json.loads(response.text)

country =[]
followers =[]

for i,j in countries['data'][0]['values'][0]['value'].items():
    country.append(i)
    followers.append(j)
country = pd.DataFrame({'country':country,'followers':followers})
new_dic1 = country.to_csv(index=False)


###engagement day by day

profile_views = []
email_contacts = []
website_clicks = []
phone_call_clicks = []
text_message_clicks = []
get_directions_clicks = []

url_eng_day = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"metric":"profile_views,email_contacts,website_clicks,phone_call_clicks,text_message_clicks,get_directions_clicks","period":"day","access_token":""+token}
headers = {
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "/",
    'Cache-Control': "no-cache",
    'Postman-Token': "f9029cdf-4d1b-4ffd-ae00-4b379bd46a1e,f9d4f0f5-2bd6-4760-a6af-24232ace531d",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url_eng_day, headers=headers, params=querystring)
engage = json.loads(response.text)
j=0
while j <= 5:
    if engage['data'][j]['name'] == 'profile_views':
        profile_views.append(engage['data'][j])
    elif engage['data'][j]['name'] == 'email_contacts':
        email_contacts.append(engage['data'][j])
    elif engage['data'][j]['name'] == 'phone_call_clicks':
        phone_call_clicks.append(engage['data'][j])
    elif engage['data'][j]['name'] == 'text_message_clicks':
        text_message_clicks.append(engage['data'][j])
    elif engage['data'][j]['name'] == 'get_directions_clicks':
        get_directions_clicks.append(engage['data'][j])
    else:
        website_clicks.append(engage['data'][j])
    j+=1    


def data_formatting(data):
    metrics_values=[]
    end_time=[]
    for i in range(len(data)):
        for j in range(len(data[i]['values'])):
            for k in data[i]['values'][j].items():
                if tuple(k)[0]=='value':
                    metrics_values.append(tuple(k)[1])
                else:
                    end_time.append(tuple(k)[1])
    return metrics_values,end_time

profiles_views_values,end_time=data_formatting(profile_views)
email_contact_values,end_time_2=data_formatting(email_contacts)
website_clicks_values,end_time_3=data_formatting(website_clicks)
phone_call_clicks_values,end_time_4=data_formatting(phone_call_clicks)
text_message_clicks_values,end_time_5=data_formatting(text_message_clicks)
get_directions_clicks_values,end_time_6=data_formatting(get_directions_clicks)

engagement4 = pd.DataFrame({'end_time':end_time,'profile_views_values':profiles_views_values,'email_contact_values':email_contact_values,'website_clicks_values':website_clicks_values,'phone_call_clicks_values':phone_call_clicks_values,'text_message_clicks_values':text_message_clicks_values,'get_directions_clicks_values':get_directions_clicks_values})
engagement4['end_time']=pd.to_datetime(engagement4['end_time']).dt.date
engagement4 = engagement4[-1:]
new_dic9 = engagement4.to_csv(index=False)


# Finding the gender and age of the followers


url_gender_age = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"metric":"audience_gender_age","period":"lifetime","access_token":""+token}
headers_gender_age = {
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url_gender_age, headers=headers_gender_age, params=querystring)
gender= json.loads(response.text)

gender_age = []
followers = []

for i,j in gender['data'][0]['values'][0]['value'].items():
    gender_age.append(i)
    followers.append(j)

gender_df = pd.DataFrame({'gender_age':gender_age,'followers':followers})
gender_df['gender'] = gender_df['gender_age'].str.split(".").str[0]
gender_df['age'] = gender_df['gender_age'].str.split(".").str[1]
gender_df.drop(['gender_age'], axis=1,inplace= True)
gender_df.sort_values(by ='age',ascending=True, inplace=True)
gender_df.reset_index(drop=True, inplace=True)
new_dic2 = gender_df.to_csv(index=False)


# # Getting the metadata of the post

url_media = "https://graph.facebook.com/v4.0/{}/media".format(insta_id)
querystring = {"fields":"id,caption,media_type,like_count,comments_count,timestamp","limit":"5000","access_token":""+token}
headers = {
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url_media, headers=headers, params=querystring)

media = json.loads(response.text)
unique = [i['id'] if 'id' in i else pd.np.NaN for i in media['data']]
caption = [i['caption'] if 'caption' in i else pd.np.NaN for i in media['data']]
media_type = [i['media_type'] if 'media_type' in i else pd.np.NaN for i in media['data']]
likes = [i['like_count'] if 'like_count' in i else pd.np.NaN for i in media['data']]
comments = [i['comments_count'] if 'comments_count' in i else pd.np.NaN for i in media['data']]
dates = [i['timestamp'] if 'timestamp' in i else pd.np.NaN for i in media['data']]
media_user = pd.DataFrame({'id':unique,'caption':caption,'media_type':media_type,'likes':likes,'comments':comments,'dates':dates})
media_user['dates']=pd.to_datetime(media_user['dates']).dt.date


# for appending the posts first we have to get the ids of all the posts

url_media = "https://graph.facebook.com/v5.0/{}/media".format(insta_id)
querystring = {"fields":"id","limit":"5000","access_token":""+token}
headers = {
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url_media, headers=headers, params=querystring)

media_id = json.loads(response.text)
post = []
for i in range(len(media_id['data'])):
    for j,k in media_id['data'][i].items():
        post.append(k)

id = post
reach = []
i=0
while i < len(id):
    url = "https://graph.facebook.com/v5.0/"+id[i]+"/insights"
    querystring = {"metric":"reach,impressions,saved","access_token":""+token}
    headers = {
        'User-Agent': "PostmanRuntime/7.16.3",
        'Accept': "*/*",
        'Cache-Control': "no-cache",
        'Postman-Token': "05dbd8f4-7bd6-489a-9832-052cf4df3cda,a19d287f-0091-4ad1-8b9d-c53acae8642c",
        'Host': "graph.facebook.com",
        'Accept-Encoding': "gzip, deflate",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    reach.append(json.loads(response.text))
    i+=1

reach_values=[]
impression_values=[]
saved_values=[]
id_=[]
id = []
for i in range(0,len(reach)):
    try:
        if 'name' in reach[i]['data'][0] and reach[i]['data'][0]['name'] == "reach":
            reach_values.append(reach[i]['data'][0]['values'][0]['value'])
        if 'name' in reach[i]['data'][1] and reach[i]['data'][1]['name'] == "impressions":
            impression_values.append(reach [i]['data'][1]['values'][0]['value'])
        if 'name' in reach[i]['data'][2] and reach[i]['data'][2]['name'] == "saved":
            saved_values.append(reach[i]['data'][2]['values'][0]['value'])  
        if 'id' in reach[i]['data'][0] and reach[i]['data'][1]['name'] == "impressions":
                id_.append(reach[i]['data'][0]['id']) 
    except:
        pass

for i in id_:
    id.append(i.split('/')[0])

engage = pd.DataFrame({'id':id,'reach_values':reach_values,'impression_values':impression_values,'saved_values':saved_values})
engagement_ = pd.merge(media_user,engage, on ='id')
engagement_['caption'] = engagement_['caption'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
engagement_['caption'] = engagement_['caption'].astype(str)
engagement_['caption']= engagement_['caption'].apply(lambda x:x.replace("\n"," ")) 
engagement_['caption']=engagement_.caption.apply(lambda x:x.replace(","," "))
engagement_['caption'] = engagement_.caption.apply(lambda x:x.strip())

new_dicccc = engagement_.to_csv(index=False)


# for videos

videos = engagement_[engagement_['media_type']=='VIDEO']
vid_id = []
for row in videos.iterrows():
    if row[1]['media_type'] == 'VIDEO':
        vid_id.append(row[1]['id'])

video = []
i=0
while i < len(vid_id):
    url = "https://graph.facebook.com/v5.0/"+vid_id[i]+"/insights"
    querystring = {"metric":"video_views","access_token":""+token}

    headers = {
        'User-Agent': "PostmanRuntime/7.19.0",
        'Accept': "*/*",
        'Cache-Control': "no-cache",
        'Postman-Token': "d7a84e90-b03d-45ee-8be2-31e3474658ea,1081a861-ebc5-4a77-add0-16909ec06e80",
        'Host': "graph.facebook.com",
        'Accept-Encoding': "gzip, deflate",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    video.append(json.loads(response.text))
    i += 1

views = []
for i in range(len(video)):
    for j,k in video[i]['data'][0]['values'][0].items():
        views.append(k)   

videos['views'] = views
new_vid = videos.to_csv(index=False)


#   page_per_day

reach = []
url = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"metric":"reach,impressions","period":"day","access_token":""+token}

headers = {
    'User-Agent': "PostmanRuntime/7.16.3",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "f4b944ee-4e15-4813-99da-212e1c9b4fed,5200d559-e9bc-410d-9f72-b951627b9a91",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
page = json.loads(response.text)
reach.append(page)

reach_values =[]
impression_values =[]
dates =[]
date =[]
for i in range(0,len(reach)):
    try:
        if 'name' in reach[i]['data'][0] and reach[i]['data'][0]['name'] == "reach":
            for j in range(len(reach[i]['data'][0]['values'])):
                reach_values.append(reach[i]['data'][0]['values'][j]['value'])
                dates.append(reach[i]['data'][0]['values'][j]['end_time'])
        if 'name' in reach[i]['data'][1] and reach[i]['data'][1]['name'] == "impressions":
            for j in range(len(reach[i]['data'][0]['values'])):
                impression_values.append(reach[i]['data'][1]['values'][j]['value'])

    except:
        pass

for i in dates:
    date.append(i.split('T')[0])

page = pd.DataFrame({'date':date,'reach_values':reach_values,'impression_values':impression_values})
page = page.iloc[-1:]
new_page = page.to_csv(index=False)



# # followers_per_day

datas = []
url = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"limit":"28","period":"day","metric":"follower_count","access_token":""+token}
headers = {
    'User-Agent': "PostmanRuntime/7.16.3",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }


response = requests.request("GET", url, headers=headers, params=querystring)
foll_day = json.loads(response.text)
datas.append(foll_day)

foll_cnt = []
date = []
dates = []

for i in range(len(datas)):
    for j in range(len(datas[i]['data'][0]['values'])):
        for k,l in datas[i]['data'][0]['values'][j].items():
            if k == 'value':
                foll_cnt.append(l)
            elif k == 'end_time':
                date.append(l)

for i in date:
    dates.append(i.split('T')[0])

foll_day_by_day = pd.DataFrame({'dates':dates,'foll_cnt':foll_cnt})
foll_day_by_day = foll_day_by_day[-1:]

url = "https://graph.facebook.com/v5.0/{}".format(insta_id)
querystring = {"fields":"followers_count","access_token":""+token}

headers = {
    'User-Agent': "PostmanRuntime/7.19.0",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "2ad63eec-a92b-41b1-9c28-d2771889e2da,f1da0ca8-a1fc-446f-bd89-078ff563e2ea",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
followers = json.loads(response.text)
foll_growth =[]
for i,j in followers.items():
    if i == 'followers_count':
        foll_growth.append(j)

foll_day_by_day['foll_on_date'] = foll_growth
new_dic7 = foll_day_by_day.to_csv(index=False)

# # active_last_4days


n3 = 4
date_3_days_ago = dt.datetime.now() - timedelta(days=n3)
date_3_days_ago = date_3_days_ago.strftime('%Y-%m-%d %H:%M:%S')
n5 = 5
date_5_days_ago = dt.datetime.now() - timedelta(days=n5)
date_5_days_ago = date_5_days_ago.strftime('%Y-%m-%d %H:%M:%S')

date_time = date_5_days_ago 
pattern = '%Y-%m-%d %H:%M:%S'
epoch1 = int(t.mktime(t.strptime(date_time, pattern)))

date_time = date_3_days_ago
pattern = '%Y-%m-%d %H:%M:%S'
epoch2 = int(t.mktime(t.strptime(date_time, pattern)))

date = [epoch1,epoch2]
url = "https://graph.facebook.com/v4.0/{}/insights".format(insta_id)
querystring = {"metric":"online_followers","period":"lifetime","since":str(date[0]),"until":str(date[1]),"access_token":""+token}
headers = {
    'User-Agent': "PostmanRuntime/7.16.3",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Host': "graph.facebook.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
followers = json.loads(response.text)
time = []
acti_mem = []
dates = []
for i in range(len(followers['data'][0]['values'])):
    for j,k in followers['data'][0]['values'][i]['value'].items():
        time.append(j)
        acti_mem.append(k)
        for j,k in followers['data'][0]['values'][i].items():
            if j =='end_time':
                dates.append(k)
followers_ = pd.DataFrame({'time':time,'acti_mem':acti_mem,'dates':dates})
followers_['time']=followers_['time'].astype(int)
followers_.sort_values(by=['dates','time'],inplace=True)

l=[]
for i in followers_['time']:
    if i > 12:
        l.append(str(i-12) +"pm")
    elif i == 0:
        l.append("12am")
    elif i == 12:
        l.append(str(i) + "pm")
    else:
        l.append(str(i)+"am")

followers_['time'] = l
date = []
for i in followers_['dates']:
    date.append(i.split("T")[0])
followers_['dates'] = date
followers_['dates'] = pd.to_datetime(followers_['dates'])
followers_['day_of_week'] = followers_['dates'].dt.day_name()
new_dicfol = followers_.to_csv(index=False)